In [1]:
%load_ext autoreload
%autoreload 2

from enumerate_datasets import *
from import_csv import *

In [2]:
dataset_paths = enumerate_datasets('.')
print('%d datasets found' %len(dataset_paths))

2 datasets found


In [3]:
for path in dataset_paths:
    import_csv(path)

In [4]:
def cursor_for_filter(path, filter):
    con = sqlite3.connect(path.replace('.csv', '.db'))
    c = con.cursor()
    c.execute(filter)
    return c

In [14]:
accumulator = 0;
count = 0;

In [16]:
def reset_gvars():
    global accumulator
    accumulator = 0;
    global count
    count = 0;

In [11]:
#calculate grand mean

for path in dataset_paths:
    print path
    c = cursor_for_filter(path, """SELECT result FROM data""")
    for value in c.fetchall():
        accumulator += value[0]
        count += 1

gm = accumulator / count
print ('grand mean is %f' %gm)

./op1.csv
-2.599
-18.21
-15.218
3.105
-0.272
-2.586
2.54
-10.616
-15.743
-2.958
./op2.csv
-5.528
1.566
-3.805
5.464
6.756
-9.071
11.635
-11.929
8.022
-7.669
grand mean is -3.355800


In [17]:
#calculate same OP means
reset_gvars()
opm = []

for idx, path in enumerate(dataset_paths):
    c = cursor_for_filter(path, """SELECT result FROM data""")

    for value in c.fetchall():
        accumulator += value[0]
        count += 1

    opm.append(accumulator / count) 
    print ('op mean is %f' %opm[-1])
    reset_gvars()

0
-2.599
-18.21
-15.218
3.105
-0.272
-2.586
2.54
-10.616
-15.743
-2.958
op mean is -6.255700
0
-5.528
1.566
-3.805
5.464
6.756
-9.071
11.635
-11.929
8.022
-7.669
op mean is -0.455900


In [28]:
#calculate mean for same part ID; need to query across dbs
reset_gvars()
snm = []

c = cursor_for_filter(dataset_paths[0], """select distinct sn, count(sn) as CountOf from data group by sn""")
sns = c.fetchall()
num_sn = len(sns)

for sn in sns:
    for idx, path in enumerate(dataset_paths):
        c = cursor_for_filter(path, """select result from data where sn = %s""" %sn[0])
        for value in c.fetchall():
            accumulator += value[0]
            count += 1
    snm.append(accumulator / count)
    reset_gvars()

print snm


[-6.19275, -2.6134999999999997, -1.2932499999999998, -2.0925000000000002, -4.587]


In [34]:
'''calculate mean for the same function: 
all measurements with the same part and operator ID’s;
represents repeatability.'''
reset_gvars()
fm = []
fm_part = []

for path in dataset_paths:
    for sn in sns:
        c = cursor_for_filter(path, """select result from data where sn = %s""" %sn[0])
        for value in c.fetchall():
            accumulator += value[0]
            count += 1
        fm_part.append(accumulator / count)
        reset_gvars()
        
    fm.append(fm_part)
    fm_part = []
    reset_gvars()

print fm

[[-10.4045, -6.0565, -1.4289999999999998, -4.038, -9.3505], [-1.9809999999999999, 0.8295000000000001, -1.1574999999999998, -0.14700000000000024, 0.17650000000000032]]


In [37]:
#calculate square difference btw means
def ss(x):
    return pow(x - gm, 2)

opss = map(ss, opm)
snss = map(ss, snm)

fss_part = []
for i in fss:
    fss_part = map(ss, i
                )
fss = map(ss, fm)

TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [9]:
#define interface at this step